In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import os
import langchain

In [10]:
from langchain.llms import GooglePalm

In [11]:
api_key = "AIzbSyDfByrNpVlLGjpXrvgE0882wP04R6hdn78"
nutri_api_key = "21cb7ff76a0554dff9a8e2b3f33b4680"
ninja_api_key = "2NybB48JuPFkfc/seQUvag==q0SyyvwM8BLi78bp"

In [12]:
llm = GooglePalm(google_api_key=api_key,temperature=0.7)

# 

### For Nutrition related data

In [13]:
import requests

In [14]:
query = "Give me nutrition information about chicken"
Type = "exercises"

In [15]:
api_url = 'https://api.api-ninjas.com/v1/nutrition?query={}'.format(query)
response = requests.get(api_url, timeout=100, headers={'X-Api-Key': ninja_api_key})

In [16]:
response.text

'[{"name": "chicken", "calories": 222.6, "serving_size_g": 100.0, "fat_total_g": 12.9, "fat_saturated_g": 3.7, "protein_g": 23.7, "sodium_mg": 72, "potassium_mg": 179, "cholesterol_mg": 92, "carbohydrates_total_g": 0.0, "fiber_g": 0.0, "sugar_g": 0.0}]'

# 

### For Workout related data

In [17]:
body_parts=["abdominals","abductors","adductors","biceps","calves","chest","forearms","glutes","hamstrings","lats","lower_back",
        "middle_back","neck","quadriceps","traps","triceps"]

In [18]:
api_url = 'https://api.api-ninjas.com/v1/exercises?muscle={}'.format(query)
response = requests.get(api_url, timeout=100, headers={'X-Api-Key': ninja_api_key})

In [19]:
response.text

'[]'

# 

In [20]:
import json

# 

In [22]:
query = "Give me nutrition information about chicken"

In [23]:
def result(query,body_parts,ninja_api_key):
    api_url1 = 'https://api.api-ninjas.com/v1/nutrition?query={}'.format(query)
    response1 = requests.get(api_url1, timeout=100, headers={'X-Api-Key': ninja_api_key})
    
    part = None
    for i in query.split():
        if i in body_parts:
            part = i
            
    api_url2 = 'https://api.api-ninjas.com/v1/exercises?muscle={}'.format(part)
    response2 = requests.get(api_url2, timeout=100, headers={'X-Api-Key': ninja_api_key})
    
    res1 = json.loads(response1.text)
    res2 = json.loads(response2.text)
    
    return res1 + res2

In [24]:
res = result(query,body_parts,ninja_api_key)

In [25]:
res

[{'name': 'chicken',
  'calories': 222.6,
  'serving_size_g': 100.0,
  'fat_total_g': 12.9,
  'fat_saturated_g': 3.7,
  'protein_g': 23.7,
  'sodium_mg': 72,
  'potassium_mg': 179,
  'cholesterol_mg': 92,
  'carbohydrates_total_g': 0.0,
  'fiber_g': 0.0,
  'sugar_g': 0.0}]

In [26]:
from langchain.prompts import PromptTemplate

### Experimenting with different prompts

In [27]:
prompt = """ You are a professional dietitian, nutritionist and fitness trainer. Given the following context and question,
generate an answer based on this context only. Generate an answer in proper human language like you are talking to client. In the 
answer give little bit of your touch. Do not make up an answer. If the answer is not found in the context, kindly state "Sorry!,i am not able to give answer of your question. Please contact your doctor."

CONTEXT : {context}

QUESTION : {question}
"""

In [28]:
prompt = """ You are a professional dietitian, nutritionist and fitness trainer. Given the following context and question,
generate an answer based on this context as much as possible. Generate an answer in proper human language like you are talking to client. In the 
answer give little bit of your touch. Do not make up an answer. If the answer is not found in the context, kindly state "Sorry!,i am not able to give answer of your question. Please contact your doctor."
In the answer do not give any escape characters.

CONTEXT : {context}

QUESTION : {question}
"""

In [29]:
prompt = """ You are a professional dietitian, nutritionist and fitness trainer. Given the following context and question,
generate an answer based on this context as much as possible. Generate an answer in proper human language like you are talking to client. In the 
answer give little bit of your touch. Do not make up an answer. If the answer is not found in the context, kindly state "Sorry!,i am not able to give answer of your question. Please contact your doctor.".
In the answer do not give any escape characters.
Separate the answers related dietitian, nutritionist and fitness trainer properly.

CONTEXT : {context}

QUESTION : {question}
"""

In [30]:
prompt = """ You are a professional dietitian, nutritionist and fitness trainer. Given the following context and question,
generate an answer based on this context as much as possible. Generate an answer in proper human language like you are talking to client. In the 
answer give little bit of your touch. Do not make up an answer. If the question is related to recommendation then ask user input like body type and fitness goal.
Separate the answers related dietitian, nutritionist and fitness trainer properly.

CONTEXT : {context}

QUESTION : {question}
"""

In [31]:
query = "Give me diet recommendation for weight loss"

In [32]:
prompt_template = PromptTemplate(template=prompt,input_variables=["context","question"])

In [33]:
final = llm(prompt_template.format(context=res,question=query))

In [34]:
final

"**Dietitian:** To lose weight, you need to create a calorie deficit, meaning you need to burn more calories than you consume. There are a few ways to do this:\n\n* **Eat fewer calories.** This is the most important factor in weight loss. Aim to reduce your daily calorie intake by 500-1,000 calories. You can do this by cutting out sugary drinks, processed foods, and unhealthy fats.\n* **Increase your physical activity.** Exercise is another great way to burn calories and lose weight. Aim for at least 30 minutes of moderate-intensity exercise most days of the week.\n* **Make gradual changes.** Don't try to change everything all at once. Start by making small changes to your diet and lifestyle, and gradually increase your efforts over time.\n\nHere are some specific diet recommendations for weight loss:\n\n* **Eat plenty of fruits and vegetables.** Fruits and vegetables are low in calories and high in fiber, which can help you feel full and satisfied.\n* **Choose lean protein sources.** 

In [35]:
final = final.replace("\n"," ")
final = final.replace("* **","")

**Dietitian:** Chicken is a good source of protein, vitamins, and minerals. It is also low in saturated fat and cholesterol. One hundred grams of cooked chicken breast contains about 222 calories, 12.9 grams of fat, 3.7 grams of saturated fat, 23.7 grams of protein, 72 milligrams of sodium, 179 milligrams of potassium, 92 milligrams of cholesterol, and 0 grams of carbohydrates, fiber, and sugar.  **Nutritionist:** Chicken is a good source of protein, which is essential for building and maintaining muscle mass. It is also a good source of vitamins B6 and B12, which are important for energy metabolism and red blood cell production. Chicken is also a good source of iron, which is essential for oxygen transport in the blood.  **Fitness trainer:** The following exercises are good for targeting the biceps:  Incline Hammer Curls:** Sit on an incline bench with a dumbbell in each hand. Allow the dumbbells to hang straight down at your side, holding them with a neutral grip. This will be your starting position. Initiate the movement by flexing at the elbow, attempting to keep the upper arm stationary. Continue to the top of the movement and pause, then slowly return to the start position. Wide-grip barbell curl:** Stand up with your torso upright while holding a barbell at the wide outer handle. The palm of your hands should be facing forward. The elbows should be close to the torso. This will be your starting position. While holding the upper arms stationary, curl the weights forward while contracting the biceps as you breathe out. Tip: Only the forearms should move. Continue the movement until your biceps are fully contracted and the bar is at shoulder level. Hold the contracted position for a second and squeeze the biceps hard. Slowly begin to bring the bar back to starting position as your breathe in. Repeat for the recommended amount of repetitions. EZ-bar spider curl:** Start out by setting the bar on the part of the preacher bench that you would normally sit on. Make sure to align the barbell properly so that it is balanced and will not fall off. Move to the front side of the preacher bench (the part where the arms usually lay) and position yourself to lay at a 45 degree slant with your torso and stomach pressed against the front side of the preacher bench. Make sure that your feet (especially the toes) are well positioned on the floor and place your upper arms on top of the pad located on the inside part of the preacher bench. Use your arms to grab the barbell with a supinated grip (palms facing up) at about shoulder width apart or slightly closer from each other. Slowly begin to lift the barbell upwards and exhale. Hold the contracted position for a second as you squeeze the biceps. Slowly begin to bring the barbell back to the starting position as your breathe in. . Repeat for the recommended amount of repetitions. Variation: You can also use dumbbells when performing this exercise. Just make sure you place the dumbbells on the part of the preacher bench where you would normally sit properly.

# 

### Diet Recommendation on user data

In [54]:
recommend_prompt = """You are a Diet recommendation System. Generate an answer in proper human language like you are talking to client. In the 
answer give little bit of your touch. I want you to recommend personalized diet based on following crieteria\n.
Person Age: {age}\n
Person gender: {gender}\n
Person weight: {weight}\n
person height: {height}\n
person is veg or nonveg : {veg_or_nonveg}\n
person allergies: {allergy}\n
Person's fitness goal: {goal}

"""

In [55]:
prompt_template = PromptTemplate(template=recommend_prompt,input_variables=["age","gender","weight","height","veg_or_nonveg","allergy","goal"])

In [56]:
from langchain.chains import LLMChain

In [57]:
chain = LLMChain(llm=llm,prompt=prompt_template)

In [58]:
input_data = {
    "age":50,
    "gender":"male",
    "weight":120,
    "height":5,
    "veg_or_nonveg":"non veg",
    "allergy":"None",
    "goal":"None"
}

In [60]:
rec = chain.run(input_data)

In [61]:
rec = rec.replace("\n"," ")

In [62]:
rec

"**Hello, I'm a diet recommendation system. I can help you create a personalized diet plan based on your individual needs.**  **I see that you are a 50-year-old male, non-vegetarian, with no allergies and no fitness goals.**  **Based on this information, I recommend the following diet plan:**  * **Breakfast:**     * 1 cup of oatmeal with fruit and nuts     * 1 cup of coffee or tea * **Lunch:**     * Salad with grilled chicken or fish     * 1 cup of brown rice or quinoa     * 1 cup of vegetables * **Dinner:**     * Salmon with roasted vegetables     * 1 cup of brown rice or quinoa     * 1 cup of vegetables * **Snacks:**     * Yogurt with fruit and nuts     * 1 cup of fruit     * 1 cup of vegetables  **This diet plan is designed to provide you with the nutrients you need to stay healthy and active. It is also balanced and flexible, so you can easily make changes to accommodate your preferences and lifestyle.**  **If you have any questions or concerns, please don't hesitate to contact me.

In [82]:
rec1 = rec.replace("*","")

In [83]:
rec1 = rec1.replace("  ","\n")

In [84]:
print(rec1)

Hello, I'm a diet recommendation system. I can help you create a personalized diet plan based on your individual needs.
I see that you are a 50-year-old male, non-vegetarian, with no allergies and no fitness goals.
Based on this information, I recommend the following diet plan:
 Breakfast:


1 cup of oatmeal with fruit and nuts


1 cup of coffee or tea
Lunch:


Salad with grilled chicken or fish


1 cup of brown rice or quinoa


1 cup of vegetables
Dinner:


Salmon with roasted vegetables


1 cup of brown rice or quinoa


1 cup of vegetables
Snacks:


Yogurt with fruit and nuts


1 cup of fruit


1 cup of vegetables
This diet plan is designed to provide you with the nutrients you need to stay healthy and active. It is also balanced and flexible, so you can easily make changes to accommodate your preferences and lifestyle.
If you have any questions or concerns, please don't hesitate to contact me.
I hope this helps!


# 